A jupyter notebook to test the code generation module

In [2]:
import os
import json
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv

In [3]:
# Read the CSV file into a Pandas DataFrame
# print(os.getcwd())
calendar_data = pd.read_csv("../data/calendar_data.csv")
print(f"The number of calendar data is {len(calendar_data)}.")

The number of calendar data is 20.


In [4]:
calendar_data.iloc[0]['start']

'2024-02-06 09:00:00-00:00'

1. Loading the dataset and json
2. Prompt the dataset column and json question to gpt-4 to generate the python function
3. pass the generated funtion and dataframe into the test function
4. save the result into json-ansewer

In [5]:
question_index = 1 # for question test

with open("../data/question.json") as json_file:
    json_data = json.load(json_file)
    print(json_data[question_index])

{'question': 'How many events do I have scheduled for today?', 'answer': ''}


In [6]:
print(f"The total number of the question is {len(json_data)}.")
print(f"Selected question is: <{json_data[question_index]['question']}>.")

The total number of the question is 8.
Selected question is: <How many events do I have scheduled for today?>.


In [15]:
text = f"""
{json_data[question_index]['question']}
"""
PROMPT = f"""You are provided a DataFrame ```calendar_data(ID, status, summary, start, end , duration, attendees)```. This dataframe includes all you calenders.
Your task is generate python function to query this dataframe and answer the question. Output a python code block without any prefix instructionn and explaination. 
Today's date is '2024-03-12 13:02:30-03:20'.

The input have following columns:
- ID: meeting ID;
- status: meeting status, including the following status: cancelled, confirmed, tentative;
- summary: meeting or event topic;
- start: the start date of meeting, date format: YYYY-MM-DD hh:mm:ss.fff-zz:xx. for example "2024-02-05 12:00:00-00:00";
- end: the start date of meeting, date format: YYYY-MM-DD hh:mm:ss.fff-zz:xx, for example "2024-02-05 13:00:00-00:00";
- duration: meeting duration (second);
- attendees: people who attend the meeting delimited by the line terminator within 1 sentence.

Output is executable Python code by enclosing it in triple backticks:
```python
<your code here>
```



For example, the output have the following format:
```
import pandas as pd 
def query(calendar_data):
    return calendar_data[0]
answer = query(calendar_data)
```

Question to be resolved: {text} 
"""

SyntaxError: invalid syntax (2441772093.py, line 1)

In [ ]:
text = f"""
{json_data[question_index]['question']}
"""
PROMPT = f"""You are provided a Pandas dataframe named calendar_data, columns = [ID, status, summary, start, end , duration, attendees]. DataFrame calendar_data includes all you calenders.
Your task is generate python function to query this dataframe and answer the question. Output a python code block without any prefix instructionn and explaination. 
Today's date is '2024-03-12 13:02:30-03:20'.


The input have following columns:
- ID: meeting ID;
- status: meeting status, including the following status: cancelled, confirmed, tentative;
- summary: meeting or event topic;
- start: the start date of meeting, date format: YYYY-MM-DD hh:mm:ss.fff-zz:xx. for example "2024-02-05 12:00:00-00:00";
- end: the start date of meeting, date format: YYYY-MM-DD hh:mm:ss.fff-zz:xx, for example "2024-02-05 13:00:00-00:00";
- duration: meeting duration (second);
- attendees: people who attend the meeting delimited by the line terminator within 1 sentence.

Output is executable Python code by enclosing it in triple backticks:
```python
<your code here>
```



For example, the output have the following format:
```
import pandas as pd 
def query(calendar_data):
    return calendar_data[0]
```

Question to be resolved: {text} 
"""

# You are given a Pandas dataframe named students_df:
# - Columns: ['Name', 'Nationality', 'Overall Grade', 'Age', 'Major', 'GPA']
# User's Question: How to find the youngest student?

In [11]:
print(PROMPT)

You are provided a DataFrame ```calendar_data(ID, status, summary, start, end , duration, attendees)```. This dataframe includes all you calenders.
Your task is generate python function to query this dataframe and answer the question. Output a python code block without any prefix instructionn and explaination.

The input have following columns:
- ID: meeting ID;
- status: meeting status, including the following status: cancelled, confirmed, tentative;
- summary: meeting or event topic;
- start: the start date of meeting, date format: YYYY-MM-DD hh:mm:ss.fff-zz:xx. for example "2024-02-05 12:00:00-00:00";
- end: the start date of meeting, date format: YYYY-MM-DD hh:mm:ss.fff-zz:xx, for example "2024-02-05 13:00:00-00:00";
- duration: meeting duration (second);
- attendees: people who attend the meeting delimited by '
' with 1 sentence.

Output using the folowing format:
code language: python
input: calendar_data
output is a executable python block without any prefix instructionn and exp

In [12]:

MODEL = "gpt-4"
# MODEL = "gpt-3.5-turbo"

def get_completion(MODEL, PROMPT):
    client = OpenAI(
        # This is the default and can be omitted
        api_key=os.environ.get("OPENAI_API_KEY"),
    )

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": PROMPT,
            }
        ],
        model=MODEL,
    )

    # verify the output
    return chat_completion.choices[0].message.content

generated_code = get_completion(MODEL, PROMPT)
print(generated_code)

'''
import pandas as pd 

def query(calendar_data):
    today_date = pd.to_datetime('2024-03-12 13:02:30-03:20').normalize()
    scheduled_meetings = calendar_data[(calendar_data['status'] == 'confirmed') & (pd.to_datetime(calendar_data['start']).dt.normalize() == today_date)]
    return len(scheduled_meetings)

answer = query(calendar_data)
'''


In [14]:
exec(generated_code)
print(answer)

NameError: name 'answer' is not defined

In [14]:
if MODEL == "gpt-4":
    FILE_NAME = 'GPT_4_generated_code.py'
else: # MODEL = "gpt-3.5-turbo"
    FILE_NAME = 'gpt_3.5_turbo_generated_code.py'

fp = open(FILE_NAME, 'w')
# Add package dependency, eg import panda as pd 
fp.write(generated_code)
fp.close()

In [15]:
import importlib
# Import your module
import GPT_4_generated_code

# Must update the function 
importlib.reload(GPT_4_generated_code)

count_correct, count_incorrect, count_code_error = 0, 0, 0

try:
    answer = GPT_4_generated_code.query(calendar_data)
except:
    print("Something goes wrong...")
    # count how many unexecutable code OR incorrect answer we get.
    # count_code_error += 1
    pass
print(answer)

2


Manully copy the funciton and test here.

Save the result into the .json file

In [ ]:
# print(json_data)


### Error

MUST self-check the output of llm, until the output is a function without any prefix sentences !!! 

1 "\n"
    should be "\\n"
2 Date
    unexpcted prefix
2. python package, eg 
    def query():
        import panda as pd
